In [ ]:
%pip install geopandas
%pip install haversine
import math
import dataclasses
from typing import List, Tuple
from datetime import datetime, date, time
import geopandas as gpd  # Import the geopandas library


@dataclasses.dataclass
class Businesses:
    name: str
    email: str
    coordinates: Tuple[float, float]
    opening_hours: 'Business_hours'

@dataclasses.dataclass
class LostRequest:
    description: str
    email: str
    polygon: List[Tuple[float, float]]
    request_date: date
    lost_time: time

@dataclasses.dataclass
class Business_hours:
    opening_days: List[str]
    opening_time: time
    closing_time: time

def is_point_inside_polygon(point: Tuple[float, float], polygon: List[Tuple[float, float]]) -> bool:
    x, y = point
    odd_nodes = False
    j = len(polygon) - 1
    for i in range(len(polygon)):
        xi, yi = polygon[i]
        xj, yj = polygon[j]
        if yi < y and yj >= y or yj < y and yi >= y:
            if xi + (y - yi) / (yj - yi) * (xj - xi) < x:
                odd_nodes = not odd_nodes
        j = i
    return odd_nodes

def is_business_open_at_time(opening_hours: Business_hours, check_date: date, check_time: time) -> bool:
    return check_date.strftime('%A') in opening_hours.opening_days and opening_hours.opening_time <= check_time <= opening_hours.closing_time

def logic(data: List[Businesses], filter_condition: LostRequest, geojson_file_path: str) -> List[Businesses]:
    # Load the GeoJSON file as a GeoDataFrame
    gdf = gpd.read_file(geojson_file_path)

def haversine_distance(coords1, coords2):
    """
    Calculate the haversine distance between two sets of coordinates.

    :param coords1: Tuple of (latitude, longitude) for the first point.
    :param coords2: Tuple of (latitude, longitude) for the second point.
    :return: Distance in meters.
    """
    # Radius of the Earth in meters
    R = 6371000

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = math.radians(coords1[0]), math.radians(coords1[1])
    lat2, lon2 = math.radians(coords2[0]), math.radians(coords2[1])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c
    return distance
    # Filter businesses based on the location_coordinates and radius
    filtered_businesses = []
    for business in data:
        distance = haversine_distance(business.coordinates, filter_condition.location_coordinates)
        if (distance <= filter_condition.radius and
            is_business_open_at_time(business.opening_hours, filter_condition.request_date, filter_condition.lost_time)):
            # Check if the business location is within the GeoDataFrame
            point = gpd.GeoSeries([business.coordinates], crs="EPSG:4326")
            businesses_in_area = gdf[gdf.geometry.intersects(point.geometry.iloc[0])]
            if not businesses_in_area.empty:
                filtered_businesses.append(business)
    return filtered_businesses

def test_logic():
    tests = [
        {
            "input": {
                "data": [
                    Businesses(
                        email="foo@bar.zip",
                        name="foo",
                        coordinates=(52.5017, 13.4183),
                        opening_hours=Business_hours(opening_days=["Monday", "Tuesday"], opening_time=time(9, 0), closing_time=time(18, 0))
                    ),
                    Businesses(
                        email="qux@bar.zip",
                        name="qux",
                        coordinates=(52.4982, 13.4290),
                        opening_hours=Business_hours(opening_days=["Monday", "Wednesday"], opening_time=time(8, 30), closing_time=time(17, 30))
                    )
                ],
                "filter_condition": LostRequest(
                    description="I lost my wallet",
                    email="test@test.com",
                    polygon=[
                        (52.4980, 13.4170),  # Adjusting polygon coordinates to include 'foo' business
                        (52.5030, 13.4170),
                        (52.5030, 13.4200),
                        (52.4980, 13.4200)
                    ],
                    request_date=date(2023, 8, 21), #monday 21/08/2023
                    lost_time=time(10, 0)
                ),
                "geojson_file_path": "your_geojson_file.geojson"  # Replace with the actual file path
            },
            "want": [
                Businesses(
                    email="foo@bar.zip",
                    name="foo",
                    coordinates=(52.5017, 13.4183),
                    opening_hours=Business_hours(opening_days=["Monday", "Tuesday"], opening_time=time(9, 0), closing_time=time(18, 0))
                )
            ]
        }
    ]

    for i, test in enumerate(tests, 1):
        got = logic(test["input"]["data"], test["input"]["filter_condition"])
        if got == test["want"]:
            print(f"Test {i} passed!")
        else:
            print(f"Test {i} failed. Expected: {test['want']}, but got: {got}")

test_logic()